In [1]:
from matplotlib import pyplot as plt
import os
from utils import Data_Handler
from Jay_module import *
from tqdm import tqdm_notebook as tqdm
import time

In [2]:
# dh = Data_Handler()
# dh.save('./dh')
dh = Data_Handler('./dh')
output_path = './data/results'

training_amount = 10
test_amount = 10
num_iter = 10

sig = [0, 18, 22] # 별 OX 폰
user = np.array(['1','2','3','4'])

colname = ['KS_only_axis', 'KS_only_gyro', 'KS_both_axis_gyro']
index = [[0,1],[2,3,4],[0,1,2,3,4]]

# dh.datasets4sig['sig-st']['사용자번호(1~4)'][도형 번호(30개)][몇 번째?(30번)]

# 처음 5개, 마지막 5개 제거
for i in range(len(dh.datasets4sig['sig-st'])):
    for j in range(len(dh.datasets4sig['sig-st']['1'])):
        del dh.datasets4sig['sig-st'][str(i+1)][j][0:5]
        del dh.datasets4sig['sig-st'][str(i+1)][j][-5:]

In [32]:
user_ = '1'
fig = 18
idx = np.array(range(20))
r = np.random.choice(20, training_amount, replace=False)
tr_idx = idx[r]
te_idx = np.delete(idx, r)
te_idx = np.random.choice(te_idx, test_amount, replace=False)

valid_tr_ = np.array(dh.datasets4sig['sig-st'][user_][fig])[tr_idx]
valid_te = np.array(dh.datasets4sig['sig-st'][user_][fig])[te_idx]
test_te = np.append([np.array(dh.datasets4sig['sig-st'][user[np.in1d(user, user_,invert=True)][0]][fig])[te_idx], 
                    np.array(dh.datasets4sig['sig-st'][user[np.in1d(user, user_,invert=True)][1]][fig])[te_idx]],
                    np.array(dh.datasets4sig['sig-st'][user[np.in1d(user, user_,invert=True)][2]][fig])[te_idx])

# Valid TR 데이터 획별로 하나로 합치기
table = valid_tr_[0]['table']
stroke_idx = np.where((table['Up-Down'].values== 'DOWN') | (table['Up-Down'].values=='UP'))[0]
stroke_num = int(len(stroke_idx)/2)


valid_tr = {str(i+1):[] for i in range(stroke_num)}
for log in valid_tr_:
    tmp = diff_dataframe(log['table'])
    for k,v in tmp.items():
        valid_tr[k].append(v.diff().dropna())

valid_tr = {k:pd.concat(valid_tr[k]) for k in valid_tr.keys()}


stroke_idx = np.where((table['Up-Down'].values== 'DOWN') | (table['Up-Down'].values=='UP'))[0]
stroke_num = int(len(stroke_idx)/2)
stroke_dict = {}
#for i in range(stroke_num):
i = 0
tmp = table.iloc[stroke_idx[i*2:i*2+2][0]:stroke_idx[i*2:i*2+2][1]]
ind = np.ones((len(tmp.columns)), bool)
ind[[0,2]] = False
tmp = tmp.iloc[:,ind].diff().dropna()
stroke_dict[str(i+1)] = tmp.iloc[:,3:].diff().dropna()

In [36]:
def velocity(X,Y, time):
    return np.linalg.norm(X-Y)/time #* int(10**8)

In [37]:
tmp.iloc[:,:3]

,Timestamp,X-Axis,Y-Axis
1,12109231.0,-0.802460,0.000000
2,16649116.0,-3.675568,-1.623718
3,16175077.0,-6.819397,-3.774048
4,16717807.0,-33.681458,-16.109375
5,16705039.0,-39.498840,-9.863281
6,16661461.0,-36.944260,10.931091
7,16595039.0,-37.607285,34.846985
8,16776192.0,-35.774933,53.129822
9,16741385.0,-25.459122,65.543701
10,16511615.0,-12.037010,71.297180


In [38]:
tmp.iloc[:,:3].apply(lambda x: velocity(x['X-Axis'], x['Y-Axis'], x['Timestamp']),axis=1)

1     6.626843e-08
2     1.232407e-07
3     1.882742e-07
4     1.051100e-06
5     1.774049e-06
6     2.873419e-06
7     4.366020e-06
8     5.299460e-06
9     5.435800e-06
10    5.047004e-06
11    4.095425e-06
12    2.509535e-06
13    5.977055e-07
14    2.004259e-06
15    3.620640e-06
16    5.882690e-06
17    6.604190e-06
18    7.060467e-06
19    6.265310e-06
20    5.202229e-06
21    3.086134e-06
22    8.909458e-07
23    1.149403e-06
24    2.239522e-06
25    2.107365e-06
26    1.803087e-06
dtype: float64

In [31]:
np.linalg.norm(tmp['X-Axis'].iloc[5]-tmp['Y-Axis'].iloc[5])/tmp['Timestamp'].iloc[5] * int(10**8)

102.34740857426648

In [ ]:
def diff_dataframe(table):
    stroke_idx = np.where((table['Up-Down'].values== 'DOWN') | (table['Up-Down'].values=='UP'))[0]
    stroke_num = int(len(stroke_idx)/2)
    stroke_dict = {}
    for i in range(stroke_num):
        tmp = table.iloc[stroke_idx[i*2:i*2+2][0]:stroke_idx[i*2:i*2+2][1]]
        stroke_dict[str(i+1)] = tmp.iloc[:,3:].diff().dropna()
    return stroke_dict

In [ ]:
start_time = time.time()
for fig in tqdm(sig):    
    sig_df = []
    iter_df = []
    for user_ in tqdm(user, leave=False):
        for i in tqdm(range(num_iter), leave=False):
            idx = np.array(range(20))
            r = np.random.choice(20, training_amount, replace=False)
            tr_idx = idx[r]
            te_idx = np.delete(idx, r)
            te_idx = np.random.choice(te_idx, test_amount, replace=False)

            valid_tr_ = np.array(dh.datasets4sig['sig-st'][user_][fig])[tr_idx]
            valid_te = np.array(dh.datasets4sig['sig-st'][user_][fig])[te_idx]
            test_te = np.append([np.array(dh.datasets4sig['sig-st'][user[np.in1d(user, user_,invert=True)][0]][fig])[te_idx], 
                                np.array(dh.datasets4sig['sig-st'][user[np.in1d(user, user_,invert=True)][1]][fig])[te_idx]],
                                np.array(dh.datasets4sig['sig-st'][user[np.in1d(user, user_,invert=True)][2]][fig])[te_idx])
            # Valid TR 데이터 획별로 하나로 합치기
            table = valid_tr_[0]['table']
            stroke_idx = np.where((table['Up-Down'].values== 'DOWN') | (table['Up-Down'].values=='UP'))[0]
            stroke_num = int(len(stroke_idx)/2)

            valid_tr = {str(i+1):[] for i in range(stroke_num)}
            for log in valid_tr_:
                tmp = diff_dataframe(log['table'])
                for k,v in tmp.items():
                    valid_tr[k].append(v.diff().dropna())

            valid_tr = {k:pd.concat(valid_tr[k]) for k in valid_tr.keys()}

            # Test & Validation
            test_df, valid_df = [], []
            for log in test_te:
                test_df.append(KS_calculator(valid_tr, log['table']))
            test_df = pd.concat(test_df)    

            for log in valid_te:
                valid_df.append(KS_calculator(valid_tr, log['table']))
            valid_df = pd.concat(valid_df)


            def final_df(valid_df, test_df):

                def transform_df(df, valid=True):
                    df =pd.concat([df.iloc[:,idx].mean(axis=1) for idx in index], axis=1)
                    df = df.reset_index(drop=True)
                    df.columns = colname

                    if valid:
                        target_idx = pd.DataFrame({"Target":np.ones(len(df)), "idx":tr_idx})
                        return pd.concat([df, target_idx], axis=1)
                    else:
                        target_idx = pd.DataFrame({"Target":np.zeros(len(df)), 
                                                  "idx":np.tile(te_idx, len(user)-1)})
                        return pd.concat([df, target_idx], axis=1)

                final_test_df = transform_df(test_df, valid=False)
                final_valid_df = transform_df(valid_df)

                df = pd.concat([final_valid_df, final_test_df], axis=0)
                return df

            ks_df = final_df(valid_df, test_df)
            tmp_result = get_df_(ks_df)

            iter_df.append(tmp_result)


    result_df = pd.concat(iter_df)
    mean_std_df = pd.DataFrame([result_df.mean(), result_df.std()], index=["Mean","Std"])
    sig_df.append(mean_std_df)
    mean_std_df.to_csv(os.path.join(output_path, "{}_results.csv".format(fig)))


final_result = pd.concat(sig_df)
final_result.to_csv(os.path.join(output_path, "fig_pilot_results.csv"))